# User Interface of the assignment of Research Track 2 #

Before starting to use this user interface, please run:
```console
$ roslaunch rt2_assignment jupyter_ass.launch
```

## imports ##

In [1]:
import rospy
import jupyros as jr 
import ipywidgets as widgets
import sys
import os

from jupyros import ros3d
from ipywidgets import interactive_output
from geometry_msgs.msg import Twist
from rt2_assignment.srv import Command

# Import the autonomous_driving class of the package.
sys.path.append("../scripts/")
import autonomous_driving

## Initialization of the ROS node ##

In [2]:
# If ui is set to None, the autonomous_driving class does not print anything.
ui = None
rospy.init_node('driving_mode')                    # Initialize the ROS node.
ad = autonomous_driving.autonomous_driving(ui)     # Initialize the clas for autonomously drive the robot.
client = rospy.ServiceProxy("/command", Command)   # Initialize the client for the driver_assistance node.
rospy.wait_for_service("/command")                 # Waits until the driver_assistance node is ready.
publisher = jr.publish('/cmd_vel', Twist)          # Initialize a publisher to the cmd_vel topic.

## Definition of widgets ##

In [3]:
# This widget displays the three driving modalities.
driving_mode = widgets.ToggleButtons(
    options=[('Autonomous driving', 1), ('Free drive', 2), ('Driver assistance', 3)],
    description='Choose:',
    value=1,
    disabled=False,
    button_style='',
)

# This widget asks for the x goal.
x_goal = widgets.FloatText(
    value=0,
    description='x goal:',
    disabled=False,
)

# This widget asks for the y goal.
y_goal = widgets.FloatText(
    value=0,
    description='y goal:',
    disabled=False,
)

# This widget shows the possible commands for the driver_assistance node.
driver_assistance = widgets.ToggleButtons(
    options=[('Increase speed', 'w'), ('Decrease speed', 's'), ('Turn left', 'a'), ('Turn right', 'd'),
             ('Stop linear', 'x'), ('Stop angular', 'z'), ('Stop sending', '0')],
    value='0',
    disabled=False,
    button_style='',
)

## User Interface ##

### Choose one of the following driving modalities: ###

In [4]:
display(driving_mode)

ToggleButtons(description='Choose:', options=(('Autonomous driving', 1), ('Free drive', 2), ('Driver assistanc…

### Drive the robot! ###

In [5]:
def ask_server(req):
    # Function binded with the driver_assistance widget to ask the server with the proper command.
    client(ord(req))
    

def choice(mode):
    # Function binded with the driving_mode widget.
    # It shows the proper user interface for different driving modalities.
    
    # If the autonomous driving is processing a goal, send  cancel request.
    if ad.is_active is True:
        ad.cancel_goal()
    
    if mode == 1:        ### Autonomous driving mode selected. ###
        
        ask_server('1')   # Disable the driver_assistance node.
        display(x_goal)
        display(y_goal)
        interactive_output(ad.reach_goal, {'x': x_goal, 'y': y_goal})   # Ask the autonomous_driving to reach the goal.

    if mode == 2:        ### Free drive mode selected. ###
        
        ask_server('1')   # Disable the driver_assistance node.
        display(publisher)
        
    if mode == 3:        ### Driver assistance mode selected. ###
        
        ask_server('0')   # Enable the driver_assistance node.
        display(driver_assistance)
        
interactive_output(ask_server, {'req': driver_assistance})
interactive_output(choice, {'mode': driving_mode})

Output()

### This is what the robot can see: ###

In [6]:
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090") 
tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client) 
map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client, continuous=True) 
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc, tf_client=tf_client) 
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client, path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000')) 
g = ros3d.GridModel()
v.objects = [g, laser_view, map_view, path, urdf]
v

Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…